In [61]:
%run partialOrder.ipynb

....................................................................................................................
----------------------------------------------------------------------
Ran 116 tests in 1.827s

OK
....................................................................................................................
----------------------------------------------------------------------
Ran 116 tests in 1.768s

OK
....................................................................................................................
----------------------------------------------------------------------
Ran 116 tests in 1.776s

OK


In [62]:
from lark import Lark, Tree, Token
from graphviz import Digraph, Graph


class Interpreter:
    
    def is_valid(self, relation, universe):
        if not (relation.get_domain() | relation.get_codomain()) <= FiniteSet(*universe):
            raise Exception(f"Relation - {relation} not valid for universe - {universe}!")
    
    def __init__(self, variables=dict(), universe=FiniteSet(), check=False):
        if check == True:
            for value in variables.values():
                self.is_valid(value, universe)
                
        self._variables = variables
        self._universe = FiniteSet(*universe)
        self._parser = Lark('''
                        RELATION: "P".."Z"
                        SET: "A".."O"
                        ?logical_operator: special_operator
                             | logical_operator "and" special_operator  -> conjunction
                             | logical_operator "or"  special_operator  -> disjunction                   
                             | logical_operator "->"  special_operator  -> implication
                             | logical_operator "=="  special_operator  -> equal
                             | logical_operator "!="  special_operator  -> not_equal
                             | logical_operator ">"   special_operator  -> greater_than
                             | logical_operator "<"   special_operator  -> lower_than
                             | logical_operator ">="  special_operator  -> greater_equal
                             | logical_operator "<="  special_operator  -> lower_equal                   
                        ?special_operator: properties
                             | "not" logical_operator      -> negation
                             | product "is" properties     -> is
                             | product "is not" properties -> is_not
                        ?properties: product    
                             | "total_order"   -> total_order
                             | "transitive"    -> transitive
                             | "lattice"       -> lattice
                             | "reflexive"     -> reflexive
                             | "symmetric"     -> symmetric
                             | "asymmetric"    -> asymmetric
                             | "antisymmetric" -> antisymmetric
                             | "injective"     -> injective
                             | "surjective"    -> surjective
                             | "bijective"     -> bijective
                        ?product: atom
                             | product "-1"       -> inverse
                             | product "°" atom   -> composition
                             | product "&" atom   -> intersection
                             | product "|" atom   -> union
                             | product "-" atom   -> difference
                             | product "^" atom   -> symmetric_difference
                             | product "*" atom   -> multiplication
                        ?atom: "("logical_operator")"
                             | SET 
                             | RELATION
                        %import common.WS
                        %ignore WS
                 ''', start='logical_operator')
        
    def parse(self, statement):
        return self._parser.parse(statement)
        
    def set_variables(self, variables=dict(), check=False):
        if check == True:
            for value in variables.values():
                self.is_valid(value, self.get_universe())
        self._variables = variables
        
    def get_variables(self, *variables):
        if len(variables) == 0:
            return self._variables
        else:
            selected_variables = dict()
            for variable in variables:
                selected_variables[variable] = self._variables[variable]
            return selected_variables
    
    def add_variable(self, variable, value, check=False):
        if check == True:
            self.is_valid(value, self.get_universe())
        variables = self.get_variables()
        if variable in variables.keys():
            raise Exception(f"Variable - '{variable}' is already defined!")
        variables[variable] = value
        self.set_variables(variables)
        
    def set_variable(self, variable, new_value, check=False):
        if check == True:
            self.is_valid(new_value, self.get_universe())
        variables = self.get_variables()
        if variable not in variables.keys():
            raise Exception(f"Variable - '{variable}' has not been defined yet!")
        variables[variable] = new_value
        self.set_variables(variables)
        
    def set_universe(self, universe=FiniteSet(), check=False):
        if check == True:
            for relation in self.get_variables().values():
                if not (relation.get_domain() | relation.get_codomain()) <= FiniteSet(*universe):
                    raise Exception(f"Universe - {universe} not valid for relation - {relation}!") 
        self._universe = FiniteSet(*universe)
        
    def get_universe(self):
        return self._universe
        
    def constant(self,data):
        try:
            variables = self.get_variables()
            return variables[data]
        except:
            raise Exception(f"Variable - '{data}' doesn't exist!")

    def interpret(self,node):
        if isinstance(node, Tree):
            if node.data == 'negation':
                return not self.interpret(node.children[0])
            elif node.data == 'inverse':
                return self.interpret(node.children[0]).converse()
            elif len(node.children) == 2:
                left, right = node.children  
                if node.data == 'equal':
                    return self.interpret(left) == self.interpret(right)
                elif node.data == 'not_equal':
                    return self.interpret(left) != self.interpret(right)
                elif node.data == 'greater_than':
                    return self.interpret(left) > self.interpret(right)
                elif node.data == 'lower_than':
                    return self.interpret(left) < self.interpret(right)
                elif node.data == 'greater_equal':
                    return self.interpret(left) >= self.interpret(right)
                elif node.data == 'lower_equal':
                    return self.interpret(left) <= self.interpret(right)
                elif node.data == 'conjunction':
                    return self.interpret(left) and self.interpret(right)
                elif node.data == 'disjunction':
                    return self.interpret(left) or self.interpret(right)
                elif node.data == 'implication':
                    return (not self.interpret(left)) or self.interpret(right)
                elif node.data == 'intersection':
                    return self.interpret(left) & self.interpret(right)
                elif node.data == 'union':
                    return self.interpret(left) | self.interpret(right)
                elif node.data == 'difference':
                    return self.interpret(left) - self.interpret(right)
                elif node.data == 'symmetric_difference':
                    return self.interpret(left) ^ self.interpret(right)
                elif node.data == 'multiplication':
                    return self.interpret(left) * self.interpret(right)
                elif node.data == 'composition':
                    return self.interpret(left).composition(self.interpret(right))
                elif node.data == 'is' or node.data == 'is_not':
                    if right.data == 'lattice':
                        return self.interpret(left).is_lattice() if node.data == 'is' \
                                                                    else not self.interpret(left).is_lattice()
                    elif right.data == 'total_order':
                        return self.interpret(left).is_total_order() if node.data == 'is' \
                                                                    else not self.interpret(left).is_total_order()
                    elif right.data == 'reflexive':
                        return self.interpret(left).is_reflexive() if node.data == 'is' \
                                                                    else not self.interpret(left).is_reflexive()
                    elif right.data == 'symmetric':
                        return self.interpret(left).is_symmetric() if node.data == 'is' \
                                                                    else not self.interpret(left).is_symmetric()
                    elif right.data == 'asymmetric':
                        return self.interpret(left).is_asymmetric() if node.data == 'is' \
                                                                    else not self.interpret(left).is_asymmetric()
                    elif right.data == 'antisymmetric':
                        return self.interpret(left).is_antisymmetric() if node.data == 'is' \
                                                                    else not self.interpret(left).is_antisymmetric()
                    elif right.data == 'transitive':
                        return self.interpret(left).is_transitive() if node.data == 'is' \
                                                                    else not self.interpret(left).is_transitive()
                    elif right.data == 'injective':
                        return self.interpret(left).is_injective() if node.data == 'is' \
                                                                    else not self.interpret(left).is_injective()
                    elif right.data == 'surjective':
                        return self.interpret(left).is_surjective() if node.data == 'is' \
                                                                    else not self.interpret(left).is_surjective()
                    elif right.data == 'bijective':
                        return self.interpret(left).is_bijective() if node.data == 'is' \
                                                                    else not self.interpret(left).is_bijective()
                else:
                    raise Exception(f"ERROR! {node.data}")
            else:
                #pass
                raise Exception(f"ERROR! Unexpected input! {node}")
        elif isinstance(node, Token):
            node = self.constant(node.value)
            return node
        else:
            raise Exception(f"ERROR! unkonown instance {type(node)}")
            
    def node2string(self,node):
        if isinstance(node, Tree):
            if node.data == 'negation':
                return "not " + self.node2string(node.children[0])
            elif node.data == 'inverse':
                return self.node2string(node.children[0]) + "-1"
            elif len(node.children) == 2:
                left, right = node.children  
                if node.data == 'equal':
                    return "(" + self.node2string(left) + " == " + self.node2string(right) + ")"
                elif node.data == 'not_equal':
                    return "(" + self.node2string(left) + " != " + self.node2string(right) + ")"
                elif node.data == 'greater_than':
                    return "(" + self.node2string(left) + " > " + self.node2string(right) + ")"
                elif node.data == 'lower_than':
                    return "(" + self.node2string(left) + " < " + self.node2string(right) + ")"
                elif node.data == 'greater_equal':
                    return "(" + self.node2string(left) + " >= " + self.node2string(right) + ")"
                elif node.data == 'lower_equal':
                    return "(" + self.node2string(left) + " <= " + self.node2string(right) + ")"
                elif node.data == 'conjunction':
                    return "(" + self.node2string(left) + " and " + self.node2string(right) + ")"
                elif node.data == 'disjunction':
                    return "(" + self.node2string(left) + " or " + self.node2string(right) + ")"
                elif node.data == 'implication':
                    return "(" + self.node2string(left) + " -> " + self.node2string(right) + ")"
                elif node.data == 'intersection':
                    return "(" + self.node2string(left) + " & " + self.node2string(right) + ")"
                elif node.data == 'union':
                    return "(" + self.node2string(left) + " | " + self.node2string(right) + ")"
                elif node.data == 'difference':
                    return "(" + self.node2string(left) + " - " + self.node2string(right) + ")"
                elif node.data == 'symmetric_difference':
                    return "(" + self.node2string(left) + " ^ " + self.node2string(right) + ")"
                elif node.data == 'multiplication':
                    return "(" + self.node2string(left) + " * " + self.node2string(right) + ")"
                elif node.data == 'composition':
                    return "(" + self.node2string(left) + " ° " + self.node2string(right) + ")"
                elif node.data == 'is' or node.data == 'is_not':
                    if right.data == 'lattice':
                        return f"{self.node2string(left)} is lattice" if node.data == 'is' \
                                                                    else f"{self.node2string(left)} is not lattice"
                    elif right.data == 'total_order':
                        return f"{self.node2string(left)} is total_order" if node.data == 'is' \
                                                                    else f"{self.node2string(left)} is not total_order"
                    elif right.data == 'reflexive':
                        return f"{self.node2string(left)} is reflexive" if node.data == 'is' \
                                                                    else f"{self.node2string(left)} is not reflexive"
                    elif right.data == 'symmetric':
                        return f"{self.node2string(left)} is symmetric" if node.data == 'is' \
                                                                    else f"{self.node2string(left)} is not symmetric"
                    elif right.data == 'asymmetric':
                        return f"{self.node2string(left)} is asymmetric" if node.data == 'is' \
                                                                    else f"{self.node2string(left)} is not asymmetric"
                    elif right.data == 'antisymmetric':
                        return f"{self.node2string(left)} is antisymmetric" if node.data == 'is' \
                                                                    else f"{self.node2string(left)} is not antisymmetric"
                    elif right.data == 'transitive':
                        return f"{self.node2string(left)} is transitive" if node.data == 'is' \
                                                                    else f"{self.node2string(left)} is not transitive"
                    elif right.data == 'injective':
                        return f"{self.node2string(left)} is injective" if node.data == 'is' \
                                                                    else f"{self.node2string(left)} is not injective"
                    elif right.data == 'surjective':
                        return f"{self.node2string(left)} is surjective" if node.data == 'is' \
                                                                    else f"{self.node2string(left)} is not surjective"
                    elif right.data == 'bijective':
                        return f"{self.node2string(left)} is bijective" if node.data == 'is' \
                                                                    else f"{self.node2string(left)} is not bijective"
                else:
                    raise Exception(f"ERROR! {node.data}")
            else:
                #pass
                raise Exception(f"ERROR! Unexpected input! {node}")
        elif isinstance(node, Token):
            return f"{node.value}"
        else:
            raise Exception(f"ERROR! unkonown instance {type(node)}")
            
            
    def explain(self, relation_data, data, expected, type_explained, mode=0):
    
        relation, var = relation_data

        if data == "lattice":
            yield from self.why_lattice(relation, var, expected, type_explained, mode)
        elif data == "total_order":
            yield from self.why_total_order(relation, var, expected, type_explained, mode)
        elif data == "reflexive":
            yield from self.why_reflexive(relation, var, expected, type_explained, mode)
        elif data == "symmetric":
            yield from self.why_symmetric(relation, var, expected, type_explained, mode)
        elif data == "asymmetric":
            yield from self.why_asymmetric(relation, var, expected, type_explained, mode)
        elif data == "antisymmetric":
            yield from self.why_antisymmetric(relation, var, expected, type_explained, mode)
        elif data == "transitive":
            yield from self.why_transitive(relation, var, expected, type_explained, mode)
            
        elif data == "injective":
            yield from self.why_injective(relation, var, expected, type_explained, mode)
        elif data == "surjective":
            yield from self.why_surjective(relation, var, expected, type_explained, mode)
        elif data == "bijective":
            yield from self.why_bijective(relation, var, expected, type_explained, mode)
            
        elif data == 'equal':
            yield from self.why_equal(relation, var, expected, type_explained, mode)
        elif data == 'not_equal':
            yield from self.why_not_equal(relation, var, expected, type_explained, mode)
        elif data == 'greater_than':
            yield from self.why_greater_than(relation, var, expected, type_explained, mode)
        elif data == 'lower_than':
            yield from self.why_lower_than(relation, var, expected, type_explained, mode)
        elif data == 'greater_equal':
            yield from self.why_greater_equal(relation, var, expected, type_explained, mode)
        elif data == 'lower_equal':
            yield from self.why_lower_than(relation, var, expected, type_explained, mode)
            
    def why_injective(self, relation, name, expected, type_explained, mode=0, extra_label=None):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation_pretty()} is injective!"+\
                f" [domain: {relation.get_domain()}]"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} is injective!"
                yield from self.why_injective_matrix(relation, name, label)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} is injective!"
                yield from self.why_injective_graph(relation, name, label)
        else:
            visited = set()
            bad_values = set()
            for x,y in relation.get_relation():
                if y in visited:bad_values.add(y);
                else:visited.add(y);
            for y in bad_values:
                if mode == 0:
                    yield f"<<EXPLAIN - {type_explained}>> {extra_label if extra_label is not None else str('')}"+\
                    f" Relation {name}={relation.get_relation_pretty()} is not "+\
                    f"injective, because elements {set(x for x in relation.is_in_relation_with(y))}"+\
                    f" are in relation with element {y}!"
                elif mode == 1:
                    label = f"<<EXPLAIN - {type_explained}>> \n {extra_label if extra_label is not None else str('')}"+\
                    f" Relation {name} is not "+\
                    f"injective, because elements {set(x for x in relation.is_in_relation_with(y))}"+\
                    f" are in relation with element {y}!"
                    yield from self.why_injective_matrix(relation, name, label,(x,y))
                elif mode == 2:
                    label = f"<<EXPLAIN - {type_explained}>> \n {extra_label if extra_label is not None else str('')}"+\
                    f" Relation {name} is not "+\
                    f"injective, because elements {set(x for x in relation.is_in_relation_with(y))}"+\
                    f" are in relation with element {y}!"
                    yield from self.why_injective_graph(relation, name, label,(x,y))
        
    def why_surjective(self, relation, name, expected, type_explained, mode=0, extra_label=None):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation_pretty()} is surjective!"+\
                f" [domain: {relation.get_domain()}]"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} is surjective!"
                yield from self.why_surjective_matrix(relation, name, label)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} is surjective!"
                yield from self.why_surjective_graph(relation, name, label)
        else:
            bad_values = relation.get_codomain() - {y for x,y in relation.get_relation()}

            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> {extra_label if extra_label is not None else str('')}"+\
                f" Relation {name}={relation.get_relation_pretty()} is not "+\
                f"surjective, because elements {bad_values}"+\
                f" are in relation with no element!"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \n{extra_label if extra_label is not None else str('')}"+\
                f"Relation {name} is not "+\
                f"surjective, because elements {set(bad_values)}"+\
                f" are in relation with no element!"
                yield from self.why_surjective_matrix(relation, name, label, bad_values)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \n{extra_label if extra_label is not None else str('')}"+\
                f"Relation {name} is not "+\
                f"surjective, because elements {set(bad_values)}"+\
                f" are in relation with no element!"
                yield from self.why_surjective_graph(relation, name, label, bad_values)
        
    def why_bijective(self, relation, name, expected, type_explained, mode=0):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation_pretty()} is bijective!"+\
                f" [domain: {relation.get_domain()}]"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} is bijective!"
                yield from self.why_bijective_matrix(relation, name, label)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} is bijective!"
                yield from self.why_bijective_graph(relation, name, label)
        else:
            bad_values = relation.get_codomain() - {y for x,y in relation.get_relation()}
            if relation.is_injective() == False:
                extra_label = f"Relation {name} is not bijective, because {name} is not injective!\n"
                yield from self.why_injective(relation, name, expected, type_explained, mode, extra_label)     
            if relation.is_surjective() == False:
                extra_label = f"Relation {name} is not bijective, because {name} is not surjective!\n"
                yield from self.why_surjective(relation, name, expected, type_explained, mode, extra_label)       
            
    def why_lattice(self, relation, name, expected, type_explained, mode=0):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation_pretty()} is lattice!"+\
                f" [domain: {relation.get_domain()}]"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} is lattice!"
                yield from self.why_lattice_matrix(relation, name, label)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} is lattice!"
                yield from self.why_lattice_graph(relation, name, label)
        else:
            for x,y in relation.cartesian_product():
                if relation.infimum([x,y]) is None:
                    if mode == 0:
                        yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation_pretty()} is not "+\
                        f"lattice because infimum [{x},{y}] is None!"
                    elif mode == 1:
                        label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} is not "+\
                        f"lattice because infimum [{x},{y}] is None!"
                        yield from self.why_lattice_matrix(relation, name, label,(x,y))
                    elif mode == 2:
                        label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} is not "+\
                        f"lattice because infimum [{x},{y}] is None!"
                        yield from self.why_lattice_graph(relation, name, label,(x,y))
                if relation.supremum([x,y]) is None:
                    if mode == 0:
                        yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation_pretty()} is not "+\
                        f"lattice because supremum [{x},{y}] is None!"
                    elif mode == 1:
                        label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} is not "+\
                        f"lattice because supremum [{x},{y}] is None!"
                        yield from self.why_lattice_matrix(relation, name, label,(x,y))
                    elif mode == 2:
                        label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} is not "+\
                        f"lattice because supremum [{x},{y}] is None!"
                        yield from self.why_lattice_graph(relation, name, label,(x,y))
        
            
    def why_total_order(self, relation, name, expected, type_explained, mode=0):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation_pretty()} "+\
                f"is total_order! [domain: {self.get_domain()}]"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} "+\
                f"is total_order!"
                yield from self.why_total_order_matrix(relation, name, label)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} "+\
                f"is total_order!"
                yield from self.why_total_order_graph(relation, name, label)
        else:
            for x in relation.get_domain():
                for y in relation.get_domain():
                    if not (relation.contains(relation.get_relation(), (x,y)) or relation.contains(relation.get_relation(), (y,x))):
                    #if not((x,y) in relation.get_relation() or (y,x) in relation.get_relation()):
                        if mode == 0:
                            yield f"<<EXPLAIN - {type_explained}>> Relation "+\
                            f"{name}={relation.get_relation_pretty()} is not total order because "+\
                            f"both elements ({x},{y}) and ({y},{x}) are missing! [domain: {relation.get_domain()}]"
                        elif mode == 1:
                            label =f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                            f"{name} is not total order because "+\
                            f"both elements ({x},{y}) and ({y},{x}) are missing!"
                            yield from self.why_total_order_matrix(relation, name, label, (x,y))
                        elif mode == 2:
                            label =f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                            f"{name} is not total order because "+\
                            f"both elements ({x},{y}) and ({y},{x}) are missing!"
                            yield from self.why_total_order_graph(relation, name, label, (x,y))
        
            
    def why_reflexive(self, relation, name, expected, type_explained, mode=0):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> "+\
                f"Relation {name}={relation.get_relation_pretty()} is reflexive!" +\
                f"[domain: {relation.get_domain()}]"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \n"+\
                f"Relation {name} is reflexive!"
                yield from self.why_reflexive_matrix(relation, name, label)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \n"+\
                f"Relation {name} is reflexive!"
                yield from self.why_reflexive_graph(relation, name, label)
        else:
            for x in relation.get_domain():
                if not relation.contains(relation.get_relation(), (x,x)): 
                #if (x,x) not in relation.get_relation():
                    if mode == 0:
                        yield f"<<EXPLAIN - {type_explained}>> "+\
                        f"Relation {name}={relation.get_relation_pretty()} is not reflexive because " +\
                        f"element ({x},{x}) is missing ! [domain: {relation.get_domain()}]"
                    elif mode == 1:
                        label = f"<<EXPLAIN - {type_explained}>> \n"+\
                        f"Relation {name} is not reflexive because " +\
                        f"element ({x},{x}) is missing !"
                        yield from self.why_reflexive_matrix(relation, name, label, (x,x))
                    elif mode == 2:
                        label = f"<<EXPLAIN - {type_explained}>> \n"+\
                        f"Relation {name} is not reflexive because " +\
                        f"element ({x},{x}) is missing !"
                        yield from self.why_reflexive_graph(relation, name, label, (x,x))
        
    
    def why_symmetric(self, relation, name, expected, type_explained, mode=0):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation_pretty()} "+\
                f"is symmetric! [domain: {relation.get_domain()}]"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} "+\
                f"is symmetric!"
                yield from self.why_symmetric_matrix(relation, name, label)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} "+\
                f"is symmetric!"
                yield from self.why_symmetric_graph(relation, name, label)
        else:
            for x,y in relation.get_relation():
                if not relation.contains(relation.get_relation(), (y,x)): 
                #if (y,x) not in relation.get_relation():
                    if mode == 0:
                        yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation_pretty()} "+\
                        f"is not symmetric because it contains element ({x},{y}), "+\
                        f"but element ({y},{x}) is missing! [domain: {relation.get_domain()}]"
                    elif mode == 1:
                        label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} "+\
                        f"is not symmetric because it contains element ({x},{y}), "+\
                        f"but element ({y},{x}) is missing!"
                        yield from self.why_symmetric_matrix(relation, name, label, (x,y))
                    elif mode == 2:
                        label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} "+\
                        f"is not symmetric because it contains element ({x},{y}), "+\
                        f"but element ({y},{x}) is missing!"
                        yield from self.why_symmetric_graph(relation, name, label, (x,y))
        
            
    def why_asymmetric(self, relation, name, expected, type_explained, mode=0):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation_pretty()} "+\
                f"is asymmetric! [domain: {relation.get_domain()}]"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} "+\
                f"is asymmetric!"
                yield from self.why_asymmetric_matrix(relation, name, label)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} "+\
                f"is asymmetric!"
                yield from self.why_asymmetric_graph(relation, name, label)
        else:
            for x,y in relation.get_relation():
                if relation.contains(relation.get_relation(), (y,x)):
                #if (y,x) in relation.get_relation():
                    if mode == 0:
                        yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation_pretty()}"+\
                        f" is not asymmetric because "+\
                        f"it contains both elements ({x},{y}) and ({y},{x})! [domain: {relation.get_domain()}]"
                    elif mode == 1:
                        label = f"<<EXPLAIN - {type_explained}>> \nRelation {name}"+\
                        f" is not asymmetric because "+\
                        f"it contains both elements ({x},{y}) and ({y},{x})!"
                        yield from self.why_asymmetric_matrix(relation, name, label, (x,y))
                    elif mode == 2:
                        label = f"<<EXPLAIN - {type_explained}>> \nRelation {name}"+\
                        f" is not asymmetric because "+\
                        f"it contains both elements ({x},{y}) and ({y},{x})!"
                        yield from self.why_asymmetric_graph(relation, name, label, (x,y))

            
    def why_antisymmetric(self, relation, name, expected, type_explained, mode=0):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation_pretty()}"+\
                f" is antisymmetric! [domain: {relation.get_domain()}]"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} "+\
                f"is antisymmetric!"
                yield from self.why_antisymmetric_matrix(relation, name, label)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name} "+\
                f"is antisymmetric!"
                yield from self.why_antisymmetric_graph(relation, name, label)
        else:
            for x,y in relation.get_relation():
                if relation.contains(relation.get_relation(), (y,x)) and (x != y):
                #if (y,x) in relation.get_relation() and (x != y):
                    if mode == 0:
                        yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation_pretty()}"+\
                        f"is not antisymmetric because "+\
                        f"it contains both elements ({x},{y}) and ({y},{x})! [domain: {relation.get_domain()}]"
                    elif mode == 1:
                        label = f"<<EXPLAIN - {type_explained}>> \nRelation {name}"+\
                        f" is not antisymmetric because "+\
                        f"it contains both elements ({x},{y}) and ({y},{x})!"
                        yield from self.why_antisymmetric_matrix(relation, name, label, (x,y))
                    elif mode == 2:
                        label = f"<<EXPLAIN - {type_explained}>> \nRelation {name}"+\
                        f" is not antisymmetric because "+\
                        f"it contains both elements ({x},{y}) and ({y},{x})!"
                        yield from self.why_antisymmetric_graph(relation, name, label, (x,y))
            
    def why_transitive(self, relation, name, expected, type_explained, mode=0):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation {name}={relation.get_relation_pretty()}"+\
                f" is transitive! [domain: {relation.get_domain()}]"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name}"+\
                f" is transitive!"
                yield from self.why_transitive_matrix(relation, name, label)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation {name}"+\
                f" is transitive!"
                yield from self.why_transitive_graph(relation, name, label)
        else:
            for x in relation.get_domain():
                for y in relation.is_out_relation_with(x):
                    for z in relation.is_out_relation_with(y):
                        if not relation.contains(relation.get_relation(), (x,z)):
                        #if (x,z) not in relation.get_relation():
                            if mode == 0:
                                yield f"<<EXPLAIN - {type_explained}>> Relation"+\
                                f" {name}={relation.get_relation_pretty()} is not transitive because "+\
                                f"it contains elements ({x},{y}) and ({y},{z}), but element ({x},{z}) is missing! "+\
                                f"[domain: {relation.get_domain()}]"
                            elif mode == 1:
                                label = f"<<EXPLAIN - {type_explained}>> \nRelation"+\
                                f" {name} is not transitive because "+\
                                f"it contains elements ({x},{y}) and ({y},{z}), but element ({x},{z}) is missing!"
                                yield from self.why_transitive_matrix(relation, name, label, (x,y,z))
                            elif mode == 2:
                                label = f"<<EXPLAIN - {type_explained}>> \nRelation"+\
                                f" {name} is not transitive because "+\
                                f"it contains elements ({x},{y}) and ({y},{z}), but element ({x},{z}) is missing!"
                                yield from self.why_transitive_graph(relation, name, label, (x,y,z))
                            
    def why_equal(self, relation1, relation2, expected, type_explained, mode=0):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation "+\
                f"{relation1[1]}={relation1[0].get_relation_pretty()} is equal to "+\
                f"relation {relation2[1]}={relation2[0].get_relation_pretty()}!"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is equal to "+\
                f"relation {relation2[1]}={relation2[0].get_relation_pretty()}!"
                yield from self.why_equal_matrix(relation1, relation2, label, True)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is equal to "+\
                f"relation {relation2[1]}={relation2[0].get_relation_pretty()}!"
                yield from self.why_equal_graph(relation1, relation2, label, True)
        else:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation "+\
                f"{relation1[1]}={relation1[0].get_relation_pretty()} is not equal to "+\
                f"relation {relation2[1]}={relation2[0].get_relation_pretty()}!"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is not equal to "+\
                f"relation {relation2[1]}!"
                yield from self.why_equal_matrix(relation1, relation2, label, False)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is not equal to "+\
                f"relation {relation2[1]}!"
                yield from self.why_equal_graph(relation1, relation2, label, False)
        
    def why_not_equal(self, relation1, relation2, expected, type_explained, mode=0):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation "+\
                f"{relation1[1]}={relation1[0].get_relation_pretty()} is not equal to "+\
                f"relation {relation2[1]}={relation2[0].get_relation_pretty()}!"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is not equal to "+\
                f"relation {relation2[1]}!"
                yield from self.why_not_equal_matrix(relation1, relation2, label, True)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is not equal to "+\
                f"relation {relation2[1]}!"
                yield from self.why_not_equal_graph(relation1, relation2, label, True)
        else:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation "+\
                f"{relation1[1]}={relation1[0].get_relation_pretty()} is equal to "+\
                f"relation {relation2[1]}={relation2[0].get_relation_pretty()}!"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is equal to "+\
                f"relation {relation2[1]}!"
                yield from self.why_not_equal_matrix(relation1, relation2, label, False)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is equal to "+\
                f"relation {relation2[1]}!"
                yield from self.why_not_equal_graph(relation1, relation2, label, False)
        
    def why_greater_than(self, relation1, relation2, expected, type_explained, mode=0):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation "+\
                f"{relation1[1]}={relation1[0].get_relation_pretty()} is strict superset of "+\
                f"relation {relation2[1]}={relation2[0].get_relation_pretty()}!"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is strict superset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_greater_than_matrix(relation1, relation2, label, True)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is strict superset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_greater_than_graph(relation1, relation2, label, True)
        else:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation "+\
                f"{relation1[1]}={relation1[0].get_relation_pretty()} is not strict superset of "+\
                f"relation {relation2[1]}={relation2[0].get_relation_pretty()}!"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is not strict superset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_greater_than_matrix(relation1, relation2, label, False)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is not strict superset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_greater_than_graph(relation1, relation2, label, False)
                
    def why_greater_equal(self, relation1, relation2, expected, type_explained, mode=0):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation "+\
                f"{relation1[1]}={relation1[0].get_relation_pretty()} is superset of "+\
                f"relation {relation2[1]}={relation2[0].get_relation_pretty()}!"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is superset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_greater_equal_matrix(relation1, relation2, label, True)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is superset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_greater_equal_graph(relation1, relation2, label, True)
        else:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation "+\
                f"{relation1[1]}={relation1[0].get_relation_pretty()} is not superset of "+\
                f"relation {relation2[1]}={relation2[0].get_relation_pretty()}!"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is not superset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_greater_equal_matrix(relation1, relation2, label, False)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is not superset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_greater_equal_graph(relation1, relation2, label, False)
                
    def why_lower_than(self, relation1, relation2, expected, type_explained, mode=0):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation "+\
                f"{relation1[1]}={relation1[0].get_relation_pretty()} is strict subset of "+\
                f"relation {relation2[1]}={relation2[0].get_relation_pretty()}!"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is strict subset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_lower_than_matrix(relation1, relation2, label, True)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is strict subset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_lower_than_graph(relation1, relation2, label, True)
        else:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation "+\
                f"{relation1[1]}={relation1[0].get_relation_pretty()} is not strict subset of "+\
                f"relation {relation2[1]}={relation2[0].get_relation_pretty()}!"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is not strict subset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_lower_than_matrix(relation1, relation2, label, False)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is not strict subset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_lower_than_graph(relation1, relation2, label, False)
                
    def why_lower_equal(self, relation1, relation2, expected, type_explained, mode=0):
        if expected == False:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation "+\
                f"{relation1[1]}={relation1[0].get_relation_pretty()} is subset of "+\
                f"relation {relation2[1]}={relation2[0].get_relation_pretty()}!"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is subset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_lower_equal_matrix(relation1, relation2, label, True)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is subset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_lower_equal_graph(relation1, relation2, label, True)
        else:
            if mode == 0:
                yield f"<<EXPLAIN - {type_explained}>> Relation "+\
                f"{relation1[1]}={relation1[0].get_relation_pretty()} is not subset of "+\
                f"relation {relation2[1]}={relation2[0].get_relation_pretty()}!"
            elif mode == 1:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is not subset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_lower_equal_matrix(relation1, relation2, label, False)
            elif mode == 2:
                label = f"<<EXPLAIN - {type_explained}>> \nRelation "+\
                f"{relation1[1]} is not subset of "+\
                f"relation {relation2[1]}!"
                yield from self.why_lower_equal_graph(relation1, relation2, label, False)
    
                            
    def product(self, node):
        if isinstance(node, Tree):
            if len(node.children) == 1:
                left = node.children[0]
                left_data = self.product(left)
                if node.data == 'inverse':
                    return left_data[0].converse(), "(" + left_data[1] + "-1" + ")"
            else:
                left, right = node.children 
                left_data = self.product(left)
                right_data = self.product(right)
                if node.data == 'intersection':
                    return left_data[0] & right_data[0], "(" + left_data[1] + "&" + right_data[1] + ")"
                elif node.data == 'union':
                    return left_data[0] | right_data[0], "(" + left_data[1] + "|" + right_data[1] + ")"
                elif node.data == 'difference':
                    return left_data[0] - right_data[0], "(" + left_data[1] + "-" + right_data[1] + ")"
                elif node.data == 'symmetric_difference':
                    return left_data[0] ^ right_data[0], "(" + left_data[1] + "^" + right_data[1] + ")"
                elif node.data == 'multiplication':
                    return left_data[0] * right_data[0], "(" + left_data[1] + "*" + right_data[1] + ")"
                elif node.data == 'composition':
                    return left_data[0].composition(right_data[0]), "(" + left_data[1] + "°" + right_data[1] + ")"
        elif isinstance(node, Token):
            return self.get_variables()[node.value], node.value
            

    def analyze(self, node, expected, type_explained, mode=0):

        actual = self.interpret(node)

        if actual != expected:

            yield f"<<<ERROR - {type_explained}>>> {self.node2string(node)}"

            if isinstance(node, Tree):
                if len(node.children) <= 2:
                    if node.data == "negation":
                        yield from self.analyze(node.children[0], not expected, type_explained, mode)
                    
                    elif node.data == "conjunction":
                        if expected == True:
                            yield from self.analyze(node.children[0], True, type_explained, mode)
                            yield from self.analyze(node.children[1], True, type_explained, mode)
                        else:
                            if self.interpret(node.children[0]) == True:
                                yield from self.analyze(node.children[0], False, type_explained, mode)
                            if self.interpret(node.children[1]) == True:
                                yield from self.analyze(node.children[1], False, type_explained, mode)
                    elif node.data == "disjunction":
                        if expected == False:
                            yield from self.analyze(node.children[0], False, type_explained, mode)
                            yield from self.analyze(node.children[1], False, type_explained, mode)
                        else:
                            if self.interpret(node.children[0]) == False:
                                yield from self.analyze(node.children[0], True, type_explained, mode)
                            if self.interpret(node.children[1]) == False:
                                yield from self.analyze(node.children[1], True, type_explained, mode)
                    elif node.data == "implication":
                        if expected == False:
                            yield from self.analyze(node.children[0], True, type_explained, mode)
                            yield from self.analyze(node.children[1], False, type_explained, mode)
                        else:
                            if self.interpret(node.children[0]) == True:
                                yield from self.analyze(node.children[0], False, type_explained, mode)
                            if self.interpret(node.children[1]) == False:
                                yield from self.analyze(node.children[1], True, type_explained, mode)
                                
                    elif node.data == 'is' or node.data == 'is_not':
                        if node.data == 'is_not': expected = not expected;
                        yield from self.explain(self.product(node.children[0]), node.children[1].data, expected, \
                                                type_explained, mode)
                    
                    else:
                        yield from self.explain([self.product(node.children[0]), self.product(node.children[1])],\
                                                node.data, expected, type_explained, mode)
                             
            
    def analyze_all(self, expression, mode=0):
    
        node = self.parse(expression)
    
        if isinstance(node, Tree):
            if node.data == "implication":
                condition, conclusion = node.children
                if self.interpret(condition) == True and self.interpret(conclusion) == False:
                    yield "Good counterexample!"
                else:
                    yield from self.analyze(condition,True,"CONDITION",mode)
                    yield from self.analyze(conclusion,False,"CONCLUSION",mode)
            else:
                raise Exception(f"ERROR! Input is NOT implication! {node}")
        else:
            raise Exception(f"ERROR! Unexpected input! {node}")
            
            
    #--------------------------------MATRIX----------------------------------------------------START  
    def why_bijective_matrix(self, relation, name, label, bad_values=None):
        dictionary = dict()
        dictionary_header = dict()
        
        if bad_values is None:
            for x in relation.get_domain(): dictionary[x] = "green";
            for y in relation.get_codomain(): dictionary_header[y] = "green";
            for x,y in relation.get_relation():
                dictionary[(x,y)] = "green"
            table = self.create_table(relation.matrix_representation(), name, dictionary, dictionary_header)
            yield from self.create_matrix(label, [(name, table)])
        else:
            pass
    
    def why_surjective_matrix(self, relation, name, label, bad_values=None):
        dictionary = dict()
        dictionary_header = dict()
        
        if bad_values is None:
            for y in relation.get_codomain(): dictionary_header[y] = "green";
            for x,y in relation.get_relation():
                dictionary[(x,y)] = "green"
            table = self.create_table(relation.matrix_representation(), name, dictionary, dictionary_header)
            yield from self.create_matrix(label, [(name, table)])
        else:
            for b in bad_values:
                dictionary_header[b] = "red"
                for x in relation.get_domain():
                    dictionary[(x,b)] = "red"
            table = self.create_table(relation.matrix_representation(), name, dictionary, dictionary_header)
            yield from self.create_matrix(label, [(name, table)])
    
    def why_injective_matrix(self, relation, name, label, bad_value=None):
        dictionary = dict()
        dictionary_header = dict()
        
        if bad_value is None:
            for x,y in relation.get_relation():
                dictionary[(x,y)] = "green"
                dictionary[x] = "green"
            table = self.create_table(relation.matrix_representation(), name, dictionary, dictionary_header)
            yield from self.create_matrix(label, [(name, table)])
        else:
            a,b = bad_value
            dictionary_header[b] = "red"
            for x,y in relation.get_relation():
                if b == y:
                    dictionary[(x,y)] = "red"
                    dictionary[x] = "red"
            table = self.create_table(relation.matrix_representation(), name, dictionary, dictionary_header)
            yield from self.create_matrix(label, [(name, table)])
    
    def why_lattice_matrix(self, relation, name, label, bad_value=None):
        dictionary = dict()
        dictionary_header = dict() 
        
        if bad_value is None:
            table = self.create_table(relation.matrix_representation(), name, dictionary)
            yield from self.create_matrix(label, [(name, table)])
        else:
            x,y = bad_value
            dictionary[x] = "red"
            dictionary[y] = "red"
            dictionary_header[x] = "red"
            dictionary_header[y] = "red"
            table = self.create_table(relation.matrix_representation(), name, dictionary, dictionary_header)
            yield from self.create_matrix(label, [(name, table)])
    
    def why_total_order_matrix(self, relation, name, label, bad_value=None):
        dictionary = dict()
        
        if bad_value is None:
            for x,y in relation.get_relation():
                dictionary[(x,y)] = "green"
            table = self.create_table(relation.matrix_representation(), name, dictionary)
            yield from self.create_matrix(label, [(name, table)])
        else:
            x,y = bad_value
            dictionary[(x,y)] = "red"
            dictionary[(y,x)] = "red"
            table = self.create_table(relation.matrix_representation(), name, dictionary)
            yield from self.create_matrix(label, [(name, table)])
    
    def why_reflexive_matrix(self, relation, name, label, bad_value=None):
        dictionary = dict()
        
        if bad_value is None:
            for x in relation.get_domain():
                dictionary[(x,x)] = "green"
            table = self.create_table(relation.matrix_representation(), name, dictionary)
            yield from self.create_matrix(label, [(name, table)])
        else:
            dictionary[bad_value] = "red"
            table = self.create_table(relation.matrix_representation(), name, dictionary)
            yield from self.create_matrix(label, [(name, table)])
            
    def why_symmetric_matrix(self, relation, name, label, bad_value=None):
        dictionary = dict()
        
        if bad_value is None:
            for x,y in relation.get_relation():
                if x != y:
                    dictionary[(x,y)] = "green"
            table = self.create_table(relation.matrix_representation(), name, dictionary)
            yield from self.create_matrix(label, [(name, table)])
        else:
            x,y = bad_value
            dictionary[(x,y)] = "red"
            dictionary[(y,x)] = "red"
            table = self.create_table(relation.matrix_representation(), name, dictionary)
            yield from self.create_matrix(label, [(name, table)])
            
    def why_asymmetric_matrix(self, relation, name, label, bad_value=None):
        dictionary = dict()
        
        if bad_value is None:
            for x,y in relation.get_relation():
                dictionary[(x,y)] = "green"
                dictionary[(y,x)] = "green"
            table = self.create_table(relation.matrix_representation(), name, dictionary)
            yield from self.create_matrix(label, [(name, table)])
        else:
            x,y = bad_value
            dictionary[(x,y)] = "red"
            dictionary[(y,x)] = "red"
            table = self.create_table(relation.matrix_representation(), name, dictionary)
            yield from self.create_matrix(label, [(name, table)])
            
    def why_antisymmetric_matrix(self, relation, name, label, bad_value=None):
        dictionary = dict()
        
        if bad_value is None:
            for x,y in relation.get_relation():
                dictionary[(x,y)] = "green"
                dictionary[(y,x)] = "green"
            table = self.create_table(relation.matrix_representation(), name, dictionary)
            yield from self.create_matrix(label, [(name, table)])
        else:
            x,y = bad_value
            dictionary[(x,y)] = "red"
            dictionary[(y,x)] = "red"
            table = self.create_table(relation.matrix_representation(), name, dictionary)
            yield from self.create_matrix(label, [(name, table)])
            
    def why_transitive_matrix(self, relation, name, label, bad_value=None):
        dictionary = dict()
        
        if bad_value is None:
            table = self.create_table(relation.matrix_representation(), name, dictionary)
            yield from self.create_matrix(label, [(name, table)])
        else:
            x,y,z = bad_value
            dictionary[(x,y)] = "green"
            dictionary[(y,z)] = "green"
            dictionary[(x,z)] = "red"
            table = self.create_table(relation.matrix_representation(), name, dictionary)
            yield from self.create_matrix(label, [(name, table)])
    
    def why_equal_matrix(self, relation1, relation2, label, equal):
        dictionary1 = dict()
        dictionary2 = dict()
        
        if equal == True:
            for p in relation1[0].get_relation():
                dictionary1[p] = "green"
                dictionary2[p] = "green"
            table1 = self.create_table(relation1[0].matrix_representation(), relation1[1], dictionary1)
            table2 = self.create_table(relation2[0].matrix_representation(), relation2[1], dictionary2)
            yield from self.create_matrix(label, [(relation1[1], table1), (relation2[1]+" ", table2)])
        else:
            for p in relation1[0].get_relation():
                if relation2[0].contains(relation2[0].get_relation(), p):
                    dictionary1[p] = "green"
                    dictionary2[p] = "green"
                else:
                    dictionary1[p] = "red"
                    dictionary2[p] = "red"
            for p in relation2[0].get_relation():
                if not relation1[0].contains(relation1[0].get_relation(), p):
                    dictionary1[p] = "red"
                    dictionary2[p] = "red"
            table1 = self.create_table(relation1[0].matrix_representation(), relation1[1], dictionary1)
            table2 = self.create_table(relation2[0].matrix_representation(), relation2[1], dictionary2)
            yield from self.create_matrix(label, [(relation1[1], table1), (relation2[1]+" ", table2)])
            
    def why_not_equal_matrix(self, relation1, relation2, label, not_equal):
        dictionary1 = dict()
        dictionary2 = dict()
        
        if not_equal == True:
            for p in relation1[0].get_relation():
                if relation2[0].contains(relation2[0].get_relation(), p):
                    dictionary1[p] = "green"
                    dictionary2[p] = "green"
                else:
                    dictionary1[p] = "red"
                    dictionary2[p] = "red"
            for p in relation2[0].get_relation():
                if not relation1[0].contains(relation1[0].get_relation(), p):
                    dictionary1[p] = "red"
                    dictionary2[p] = "red"
            table1 = self.create_table(relation1[0].matrix_representation(), relation1[1], dictionary1)
            table2 = self.create_table(relation2[0].matrix_representation(), relation2[1], dictionary2)
            yield from self.create_matrix(label, [(relation1[1], table1), (relation2[1]+" ", table2)])
        else:
            for p in relation1[0].get_relation():
                dictionary1[p] = "green"
                dictionary2[p] = "green"
            table1 = self.create_table(relation1[0].matrix_representation(), relation1[1], dictionary1)
            table2 = self.create_table(relation2[0].matrix_representation(), relation2[1], dictionary2)
            yield from self.create_matrix(label, [(relation1[1], table1), (relation2[1]+" ", table2)])
            
            
    def why_greater_than_matrix(self, relation1, relation2, label, greater):
        dictionary1 = dict()
        dictionary2 = dict()
        
        if greater == True:
            for p in relation1[0].get_relation():
                dictionary1[p] = "YellowGreen"
            for p in relation2[0].get_relation():
                dictionary1[p] = "green"
                dictionary2[p] = "green"
            table1 = self.create_table(relation1[0].matrix_representation(), relation1[1], dictionary1)
            table2 = self.create_table(relation2[0].matrix_representation(), relation2[1], dictionary2)
            yield from self.create_matrix(label, [(relation1[1], table1), (relation2[1]+" ", table2)])
        else:
            for p in relation2[0].get_relation():
                if relation1[0].contains(relation1[0].get_relation(), p):
                    dictionary1[p] = "green"
                    dictionary2[p] = "green"
                else:
                    dictionary1[p] = "red"
                    dictionary2[p] = "red"
            table1 = self.create_table(relation1[0].matrix_representation(), relation1[1], dictionary1)
            table2 = self.create_table(relation2[0].matrix_representation(), relation2[1], dictionary2)
            yield from self.create_matrix(label, [(relation1[1], table1), (relation2[1]+" ", table2)])

    def why_greater_equal_matrix(self, relation1, relation2, label, greater):
        dictionary1 = dict()
        dictionary2 = dict()
        
        if greater == True:
            for p in relation1[0].get_relation():
                dictionary1[p] = "YellowGreen"
            for p in relation2[0].get_relation():
                dictionary1[p] = "green"
                dictionary2[p] = "green"
            table1 = self.create_table(relation1[0].matrix_representation(), relation1[1], dictionary1)
            table2 = self.create_table(relation2[0].matrix_representation(), relation2[1], dictionary2)
            yield from self.create_matrix(label, [(relation1[1], table1), (relation2[1]+" ", table2)])
        else:
            for p in relation2[0].get_relation():
                if relation1[0].contains(relation1[0].get_relation(), p):
                    dictionary1[p] = "green"
                    dictionary2[p] = "green"
                else:
                    dictionary1[p] = "red"
                    dictionary2[p] = "red"
            table1 = self.create_table(relation1[0].matrix_representation(), relation1[1], dictionary1)
            table2 = self.create_table(relation2[0].matrix_representation(), relation2[1], dictionary2)
            yield from self.create_matrix(label, [(relation1[1], table1), (relation2[1]+" ", table2)])
            
    def why_lower_than_matrix(self, relation1, relation2, label, lower):
        dictionary1 = dict()
        dictionary2 = dict()
        
        if lower == True:
            for p in relation2[0].get_relation():
                dictionary2[p] = "YellowGreen"
            for p in relation1[0].get_relation():
                dictionary1[p] = "green"
                dictionary2[p] = "green"
            table1 = self.create_table(relation1[0].matrix_representation(), relation1[1], dictionary1)
            table2 = self.create_table(relation2[0].matrix_representation(), relation2[1], dictionary2)
            yield from self.create_matrix(label, [(relation1[1], table1), (relation2[1]+" ", table2)])
        else:
            for p in relation1[0].get_relation():
                if relation2[0].contains(relation2[0].get_relation(), p):
                    dictionary1[p] = "green"
                    dictionary2[p] = "green"
                else:
                    dictionary1[p] = "red"
                    dictionary2[p] = "red"
            table1 = self.create_table(relation1[0].matrix_representation(), relation1[1], dictionary1)
            table2 = self.create_table(relation2[0].matrix_representation(), relation2[1], dictionary2)
            yield from self.create_matrix(label, [(relation1[1], table1), (relation2[1]+" ", table2)])

    def why_lower_equal_matrix(self, relation1, relation2, label, lower):
        dictionary1 = dict()
        dictionary2 = dict()
        
        if lower == True:
            for p in relation2[0].get_relation():
                dictionary2[p] = "YellowGreen"
            for p in relation1[0].get_relation():
                dictionary1[p] = "green"
                dictionary2[p] = "green"
            table1 = self.create_table(relation1[0].matrix_representation(), relation1[1], dictionary1)
            table2 = self.create_table(relation2[0].matrix_representation(), relation2[1], dictionary2)
            yield from self.create_matrix(label, [(relation1[1], table1), (relation2[1]+" ", table2)])
        else:
            for p in relation1[0].get_relation():
                if relation2[0].contains(relation2[0].get_relation(), p):
                    dictionary1[p] = "green"
                    dictionary2[p] = "green"
                else:
                    dictionary1[p] = "red"
                    dictionary2[p] = "red"
            table1 = self.create_table(relation1[0].matrix_representation(), relation1[1], dictionary1)
            table2 = self.create_table(relation2[0].matrix_representation(), relation2[1], dictionary2)
            yield from self.create_matrix(label, [(relation1[1], table1), (relation2[1]+" ", table2)])
    
    def new_header(self, values, highlight=None):
        
        row = "<TR><TD color=\"white\"></TD>"
            
        for p in values:
            if highlight is not None and p in highlight.keys():
                row += f"<TD bgcolor=\"{highlight[p]}\">{p}</TD>"
            else:
                row += f"<TD color=\"white\">{p}</TD>"
            
        row += "</TR>"
        
        return row
            
    def new_row(self, values, head, codom, highlight):
        
        row = f"<TR><TD color=\"white\">{head}</TD>"
        
        if head in highlight.keys():
            row = f"<TR><TD bgcolor=\"{highlight[head]}\">{head}</TD>"
            
        for i in range(len(codom)):
            
            if (head, codom[i]) not in highlight.keys():
                row += f"<TD color=\"black\">{values[i]}</TD>"
            else:
                row += f"<TD color=\"black\" bgcolor=\"{highlight[(head, codom[i])]}\">{values[i]}</TD>"
            
        row += "</TR>"
        
        return row
    
    def create_table(self, values, label, dictionary=dict(), header=None):
        matrix, dom, codom = values
        
        r = "<<TABLE color=\"white\">"
        
        if header == None:
            r += self.new_header(codom)
        else:
            r += self.new_header(codom, header)
        
        for i in range(len(matrix)):
            r += self.new_row(matrix[i], dom[i], codom, dictionary)
            
        r += f"<TR><TD colspan=\"{len(matrix)+1}\">{label}</TD></TR>"
        r += "</TABLE>>"
        return r 

    
    def create_matrix(self, label, matrices):
        
        g = Graph(node_attr={'shape': 'none'})
        
        g.body.append(f"label=\"{label}\"")
        
        for name, matrix in matrices:
            g.node(name, label=f'{matrix}')

        #g.body.append("{rank=same tab1 tab2}")
        yield g
    
    #--------------------------------MATRIX----------------------------------------------------END
    
    #--------------------------------GRAPH-----------------------------------------------------START
    
    def why_bijective_graph(self, relation, name, label, bad_values=None):
        dictionary_node = dict()
        dictionary_edge = dict()
        
        dictionary_edge["dotted"] = set()
        
        if isinstance(relation, Function):        
            graph_type = 3
            r1_d, r1_c = 1,2
        elif isinstance(relation, BinaryRelation):        
            graph_type = 0
            r1_d, r1_c = 1,2
        
        if bad_values is None:
            for x in relation.get_domain(): dictionary_node[(x,r1_d)] = "green";
            for y in relation.get_codomain(): dictionary_node[(y,r1_c)] = "green";
            for x,y in relation.get_relation():
                dictionary_edge[(x,y)] = "green"

        yield from self.create_graph(label, [(name, relation, dictionary_node, dictionary_edge)], graph_type)
    
    def why_surjective_graph(self, relation, name, label, bad_values=None):
        dictionary_node = dict()
        dictionary_edge = dict()
        
        dictionary_edge["dotted"] = set()
        
        if isinstance(relation, Function):        
            graph_type = 3
            r1_d, r1_c = 1,2
        elif isinstance(relation, BinaryRelation):        
            graph_type = 0
            r1_d, r1_c = 1,2
        
        if bad_values is None:
            for y in relation.get_codomain(): dictionary_node[(y,r1_c)] = "green";
            for x,y in relation.get_relation():
                dictionary_edge[(x,y)] = "green"
        else:
            for b in bad_values:
                dictionary_node[(b,r1_c)] = "red"
                for x in relation.get_domain():
                    dictionary_edge[(x,b)] = "red"
                    
        yield from self.create_graph(label, [(name, relation, dictionary_node, dictionary_edge)], graph_type)
    
    def why_injective_graph(self, relation, name, label, bad_value=None):
        dictionary_node = dict()
        dictionary_edge = dict()
        
        dictionary_edge["dotted"] = set()

        if isinstance(relation, Function):        
            graph_type = 3
            r1_d, r1_c = 1,2
        elif isinstance(relation, BinaryRelation):        
            graph_type = 0
            r1_d, r1_c = 1,2
        
        if bad_value is None:
            for x,y in relation.get_relation():
                dictionary_node[(x,r1_d)] = "green"
                dictionary_node[(y,r1_c)] = "green"
                dictionary_edge[(x,y)] = "green"
        else:
            a,b = bad_value
            dictionary_node[(b,r1_c)] = "red"
            for x,y in relation.get_relation():
                if b == y:
                    dictionary_node[(x,r1_d)] = "red"
                    dictionary_edge[(x,y)] = "red"
      
        yield from self.create_graph(label, [(name, relation, dictionary_node, dictionary_edge)], graph_type)
    
    def why_lattice_graph(self, relation, name, label, bad_value=None):
        dictionary_node = dict()
        dictionary_edge = dict()
        
        dictionary_edge["dotted"] = set()
        
        if isinstance(relation, PartialOrder):      
            graph_type = 2         
            r1_d, r1_c = 1,1
        
        if bad_value is None:
            pass
        else:
            x,y = bad_value
            dictionary_node[(x,r1_d)] = "red"
            dictionary_node[(y,r1_c)] = "red"
        
        yield from self.create_graph(label, [(name, relation, dictionary_node, dictionary_edge)], graph_type)
        
        
    def why_total_order_graph(self, relation, name, label, bad_value=None):
        dictionary_node = dict()
        dictionary_edge = dict()
        
        dictionary_edge["dotted"] = set()
        
        if isinstance(relation, PartialOrder):      
            graph_type = 2         
            r1_d, r1_c = 1,1
        
        if bad_value is None:
            for x,y in relation.get_relation():
                dictionary_node[(x,r1_d)] = "green"
                dictionary_node[(y,r1_c)] = "green"
        else:
            x,y = bad_value
            dictionary_node[(x, r1_d)] = "red"
            dictionary_node[(y, r1_c)] = "red"
            
        yield from self.create_graph(label, [(name, relation, dictionary_node, dictionary_edge)], graph_type)
            
    def why_reflexive_graph(self, relation, name, label, bad_value=None):
        dictionary_node = dict()
        dictionary_edge = dict()
        
        dictionary_edge["dotted"] = set()
        
        if isinstance(relation, PartialOrder):      
            graph_type = 2         
            r1_d, r1_c = 1,1
        elif isinstance(relation, HomogeneousRelation):
            graph_type = 1  
            r1_d, r1_c = 1,1
        
        if bad_value is None:
            for x in relation.get_domain():
                dictionary_node[(x, r1_d)] = "green"
                dictionary_edge[(x,x)]     = "green";
        else:
            dictionary_node[(bad_value[0], r1_d)] = "red"
            dictionary_node[(bad_value[1], r1_c)] = "red"
            dictionary_edge["dotted"].add((bad_value, "red"))
            
        yield from self.create_graph(label, [(name, relation, dictionary_node, dictionary_edge)], graph_type)
        
    def why_symmetric_graph(self, relation, name, label, bad_value=None):
        dictionary_node = dict()
        dictionary_edge = dict()
        
        dictionary_edge["dotted"] = set()
        
        if isinstance(relation, PartialOrder):      
            graph_type = 2         
            r1_d, r1_c = 1,1
        elif isinstance(relation, HomogeneousRelation):
            graph_type = 1  
            r1_d, r1_c = 1,1
        
        if bad_value is None:
            for x,y in relation.get_relation():
                if x != y:
                    dictionary_node[(x, r1_d)] = "green"
                    dictionary_node[(y, r1_c)] = "green"
                    dictionary_edge[(x,y)]     = "green"
        else:
            x,y = bad_value
            dictionary_node[(x, r1_d)] = "red"
            dictionary_node[(y, r1_c)] = "red"
            dictionary_edge["dotted"].add(((y,x), "red"))
            dictionary_edge[(x,y)] = "red"
            
        yield from self.create_graph(label, [(name, relation, dictionary_node, dictionary_edge)], graph_type)
        
    def why_asymmetric_graph(self, relation, name, label, bad_value=None):
        dictionary_node = dict()
        dictionary_edge = dict()

        if isinstance(relation, PartialOrder):      
            graph_type = 2         
            r1_d, r1_c = 1,1
        elif isinstance(relation, HomogeneousRelation):
            graph_type = 1  
            r1_d, r1_c = 1,1
        
        if bad_value is None:
            for x,y in relation.get_relation():
                dictionary_node[(x, r1_d)] = "green"
                dictionary_node[(y, r1_c)] = "green"
                dictionary_edge[(x,y)]     = "green"
        else:
            x,y = bad_value
            dictionary_node[(x, r1_d)] = "red"
            dictionary_node[(y, r1_c)] = "red"
            dictionary_edge[(x,y)] = "red"
            dictionary_edge[(y,x)] = "red"
        yield from self.create_graph(label, [(name, relation, dictionary_node, dictionary_edge)], graph_type)
    
    def why_antisymmetric_graph(self, relation, name, label, bad_value=None):
        yield from self.why_asymmetric_graph(relation, name, label, bad_value)
        
    def why_transitive_graph(self, relation, name, label, bad_value=None):
        dictionary_node = dict()
        dictionary_edge = dict()
        
        dictionary_edge["dotted"] = set()
        
        if isinstance(relation, PartialOrder):      
            graph_type = 2         
            r1_d, r1_c = 1,1
        elif isinstance(relation, HomogeneousRelation):
            graph_type = 1  
            r1_d, r1_c = 1,1
        
        if bad_value is None: 
            yield from self.create_graph(label, [(name, relation, dictionary_node, dictionary_edge)], graph_type)
        else:
            x,y,z = bad_value
            dictionary_node[(x, r1_d)] = "red"
            dictionary_node[(y, r1_c)] = "green"
            dictionary_edge[(x,y)]     = "green"
            dictionary_node[(z, r1_c)] = "red"
            dictionary_edge[(y,z)]     = "green"
            dictionary_edge["dotted"].add(((x,z), "red"))
            yield from self.create_graph(label, [(name, relation, dictionary_node, dictionary_edge)], graph_type)
    
    def why_equal_graph(self, relation1, relation2, label, equal):
        dictionary1_node = dict()
        dictionary1_edge = dict()
        dictionary2_node = dict()
        dictionary2_edge = dict()
        
        dictionary1_edge["dotted"] = set()
        dictionary2_edge["dotted"] = set()
        
        if isinstance(relation1[0], PartialOrder):      
            graph_type = 2         
            r1_d, r1_c, r2_d, r2_c = 1,1,2,2
        elif isinstance(relation1[0], HomogeneousRelation):
            graph_type = 1  
            r1_d, r1_c, r2_d, r2_c = 1,1,2,2
        elif isinstance(relation1[0], Function):        
            graph_type = 3
            r1_d, r1_c, r2_d, r2_c = 1,2,3,4
        elif isinstance(relation1[0], BinaryRelation):        
            graph_type = 0
            r1_d, r1_c, r2_d, r2_c = 1,2,3,4
        
        if equal == True:         
            for p in relation1[0].get_relation():
                    dictionary1_node[(p[0], r1_d)] = "green"
                    dictionary1_node[(p[1], r1_c)] = "green"
                    dictionary1_edge[p]         = "green"
                    dictionary2_node[(p[0], r2_d)] = "green"
                    dictionary2_node[(p[1], r2_c)] = "green"
                    dictionary2_edge[p]         = "green"           
        else:
            for p in relation1[0].get_relation():
                if relation2[0].contains(relation2[0].get_relation(), p):
                    if (p[0],r1_d) not in dictionary1_node.keys(): dictionary1_node[(p[0], r1_d)] = "green";
                    if (p[1],r1_c) not in dictionary1_node.keys(): dictionary1_node[(p[1], r1_c)] = "green";
                    dictionary1_edge[p] = "green"
                    if (p[0],r2_d) not in dictionary2_node.keys(): dictionary2_node[(p[0], r2_d)] = "green";
                    if (p[1],r2_c) not in dictionary2_node.keys(): dictionary2_node[(p[1], r2_c)] = "green";
                    dictionary2_edge[p] = "green"
                else:
                    dictionary1_node[(p[0], r1_d)] = "red"
                    dictionary1_node[(p[1], r1_c)] = "red"
                    dictionary1_edge[p] = "red"
                    dictionary2_node[(p[0], r2_d)] = "red"
                    dictionary2_node[(p[1], r2_c)] = "red"
                    dictionary2_edge["dotted"].add((p, "red"))   
            for p in relation2[0].get_relation():
                if not relation1[0].contains(relation1[0].get_relation(), p):
                    dictionary1_node[(p[0], r1_d)] = "red"
                    dictionary1_node[(p[1], r1_c)] = "red"
                    dictionary1_edge["dotted"].add((p, "red"))   
                    dictionary2_node[(p[0], r2_d)] = "red"
                    dictionary2_node[(p[1], r2_c)] = "red"
                    dictionary2_edge[p] = "red"
            
        yield from self.create_graph(label, [(relation1[1], relation1[0], dictionary1_node, dictionary1_edge), \
                                    (relation2[1], relation2[0], dictionary2_node, dictionary2_edge)], graph_type)
        
    def why_not_equal_graph(self, relation1, relation2, label, not_equal):
        dictionary1_node = dict()
        dictionary1_edge = dict()
        dictionary2_node = dict()
        dictionary2_edge = dict()
        
        dictionary1_edge["dotted"] = set()
        dictionary2_edge["dotted"] = set()
        
        if isinstance(relation1[0], PartialOrder):      
            graph_type = 2         
            r1_d, r1_c, r2_d, r2_c = 1,1,2,2
        elif isinstance(relation1[0], HomogeneousRelation):
            graph_type = 1  
            r1_d, r1_c, r2_d, r2_c = 1,1,2,2
        elif isinstance(relation1[0], Function):        
            graph_type = 3
            r1_d, r1_c, r2_d, r2_c = 1,2,3,4
        elif isinstance(relation1[0], BinaryRelation):        
            graph_type = 0
            r1_d, r1_c, r2_d, r2_c = 1,2,3,4
        
        if not_equal == False:         
            for p in relation1[0].get_relation():
                    dictionary1_node[(p[0], r1_d)] = "green"
                    dictionary1_node[(p[1], r1_c)] = "green"
                    dictionary1_edge[p]         = "green"
                    dictionary2_node[(p[0], r2_d)] = "green"
                    dictionary2_node[(p[1], r2_c)] = "green"
                    dictionary2_edge[p]         = "green"           
        else:
            for p in relation1[0].get_relation():
                if relation2[0].contains(relation2[0].get_relation(), p):
                    if (p[0],r1_d) not in dictionary1_node.keys(): dictionary1_node[(p[0], r1_d)] = "green";
                    if (p[1],r1_c) not in dictionary1_node.keys(): dictionary1_node[(p[1], r1_c)] = "green";
                    dictionary1_edge[p] = "green"
                    if (p[0],r2_d) not in dictionary2_node.keys(): dictionary2_node[(p[0], r2_d)] = "green";
                    if (p[1],r2_c) not in dictionary2_node.keys(): dictionary2_node[(p[1], r2_c)] = "green";
                    dictionary2_edge[p] = "green"
                else:
                    dictionary1_node[(p[0], r1_d)] = "red"
                    dictionary1_node[(p[1], r1_c)] = "red"
                    dictionary1_edge[p] = "red"
                    dictionary2_node[(p[0], r2_d)] = "red"
                    dictionary2_node[(p[1], r2_c)] = "red"
                    dictionary2_edge["dotted"].add((p, "red"))   
            for p in relation2[0].get_relation():
                if not relation1[0].contains(relation1[0].get_relation(), p):
                    dictionary1_node[(p[0], r1_d)] = "red"
                    dictionary1_node[(p[1], r1_c)] = "red"
                    dictionary1_edge["dotted"].add((p, "red"))   
                    dictionary2_node[(p[0], r2_d)] = "red"
                    dictionary2_node[(p[1], r2_c)] = "red"
                    dictionary2_edge[p] = "red"
            
        yield from self.create_graph(label, [(relation1[1], relation1[0], dictionary1_node, dictionary1_edge), \
                                    (relation2[1], relation2[0], dictionary2_node, dictionary2_edge)], graph_type)
        
    def why_greater_than_graph(self, relation1, relation2, label, greater):
        dictionary1_node = dict()
        dictionary1_edge = dict()
        dictionary2_node = dict()
        dictionary2_edge = dict()
        
        dictionary1_edge["dotted"] = set()
        dictionary2_edge["dotted"] = set()
        
        if isinstance(relation1[0], PartialOrder):      
            graph_type = 2         
            r1_d, r1_c, r2_d, r2_c = 1,1,2,2
        elif isinstance(relation1[0], HomogeneousRelation):
            graph_type = 1  
            r1_d, r1_c, r2_d, r2_c = 1,1,2,2
        elif isinstance(relation1[0], Function):        
            graph_type = 3
            r1_d, r1_c, r2_d, r2_c = 1,2,3,4
        elif isinstance(relation1[0], BinaryRelation):        
            graph_type = 0
            r1_d, r1_c, r2_d, r2_c = 1,2,3,4
        
        if greater == True:         
            for p in relation1[0].get_relation():
                
                if relation2[0].contains(relation2[0].get_relation(), p):
                    dictionary1_node[(p[0], r1_d)] = "green" 
                    dictionary1_node[(p[1], r1_c)] = "green" 
                    dictionary1_edge[p]            = "green" 
                    dictionary2_node[(p[0], r2_d)] = "green"
                    dictionary2_node[(p[1], r2_c)] = "green"
                    dictionary2_edge[p]            = "green" 
                else:
                    if (p[0],r1_d) not in dictionary1_node.keys():dictionary1_node[(p[0], r1_d)] = "YellowGreen";
                    if (p[1],r1_c) not in dictionary1_node.keys():dictionary1_node[(p[1], r1_c)] = "YellowGreen";
                    dictionary1_edge[p] = "YellowGreen";
                    #dictionary2["dotted"].add((p, "lightgrey"))  
                              
        else:
            for p in relation2[0].get_relation():
                if relation1[0].contains(relation1[0].get_relation(), p):
                    if (p[0],r1_d) not in dictionary1_node.keys():dictionary1_node[(p[0], r1_d)] = "green";
                    if (p[1],r1_c) not in dictionary1_node.keys():dictionary1_node[(p[1], r1_c)] = "green";
                    dictionary1_edge[p] = "green"
                    if (p[0],r2_d) not in dictionary2_node.keys():dictionary2_node[(p[0], r2_d)] = "green";
                    if (p[1],r2_c) not in dictionary2_node.keys():dictionary2_node[(p[1], r2_c)] = "green";
                    dictionary2_edge[p] = "green"
                else:
                    dictionary2_node[(p[0], r2_d)] = "red"
                    dictionary2_node[(p[1], r2_c)] = "red"
                    dictionary2_edge[p] = "red"
                    dictionary1_edge["dotted"].add((p, "red")) 
                    dictionary1_node[(p[0], r1_d)] = "red"
                    dictionary1_node[(p[1], r1_c)] = "red"        
            
        yield from self.create_graph(label, [(relation1[1], relation1[0], dictionary1_node, dictionary1_edge), \
                                    (relation2[1], relation2[0], dictionary2_node, dictionary2_edge)], graph_type)
        

    def why_greater_equal_graph(self, relation1, relation2, label, greater):
        yield from self.why_greater_than_graph(relation1, relation2, label, greater)
        
    def why_lower_than_graph(self, relation1, relation2, label, lower):
        dictionary1_node = dict()
        dictionary1_edge = dict()
        dictionary2_node = dict()
        dictionary2_edge = dict()
        
        dictionary1_edge["dotted"] = set()
        dictionary2_edge["dotted"] = set()
        
        if isinstance(relation1[0], PartialOrder):      
            graph_type = 2         
            r1_d, r1_c, r2_d, r2_c = 1,1,2,2
        elif isinstance(relation1[0], HomogeneousRelation):
            graph_type = 1  
            r1_d, r1_c, r2_d, r2_c = 1,1,2,2
        elif isinstance(relation1[0], BinaryRelation):        
            graph_type = 0
            r1_d, r1_c, r2_d, r2_c = 1,2,3,4
        
        if lower == True:         
            for p in relation2[0].get_relation():
                
                if relation1[0].contains(relation1[0].get_relation(), p):
                    dictionary1_node[(p[0], r1_d)] = "green" 
                    dictionary1_node[(p[1], r1_c)] = "green" 
                    dictionary1_edge[p]            = "green" 
                    dictionary2_node[(p[0], r2_d)] = "green"
                    dictionary2_node[(p[1], r2_c)] = "green"
                    dictionary2_edge[p]            = "green" 
                else:
                    if (p[0],r2_d) not in dictionary2_node.keys():dictionary2_node[(p[0], r2_d)] = "YellowGreen";
                    if (p[1],r2_c) not in dictionary2_node.keys():dictionary2_node[(p[1], r2_c)] = "YellowGreen";
                    dictionary2_edge[p] = "YellowGreen";
                    #dictionary1["dotted"].add((p, "lightgrey"))  
                              
        else:
            for p in relation1[0].get_relation():
                if relation2[0].contains(relation2[0].get_relation(), p):
                    if (p[0],r1_d) not in dictionary1_node.keys():dictionary1_node[(p[0], r1_d)] = "green";
                    if (p[1],r1_c) not in dictionary1_node.keys():dictionary1_node[(p[1], r1_c)] = "green";
                    dictionary1_edge[p] = "green"
                    if (p[0],r2_d) not in dictionary2_node.keys():dictionary2_node[(p[0], r2_d)] = "green";
                    if (p[1],r2_c) not in dictionary2_node.keys():dictionary2_node[(p[1], r2_c)] = "green";
                    dictionary2_edge[p] = "green"
                else:
                    dictionary2_node[(p[0], r2_d)] = "red"
                    dictionary2_node[(p[1], r2_c)] = "red" 
                    dictionary2_edge["dotted"].add((p, "red")) 
                    dictionary1_node[(p[0], r1_d)] = "red"
                    dictionary1_node[(p[1], r1_c)] = "red"
                    dictionary1_edge[p] = "red"
            
        yield from self.create_graph(label, [(relation1[1], relation1[0], dictionary1_node, dictionary1_edge), \
                                    (relation2[1], relation2[0], dictionary2_node, dictionary2_edge)], graph_type)
        

    def why_lower_equal_graph(self, relation1, relation2, label, lower):
        yield from self.why_lower_than_graph(relation1, relation2, label, lower)

    
    def create_subgraph(self, name, label, nodes, graph, graph_label, level=0, highlight=dict()):
        
        with graph.subgraph(name=f"cluster_{graph_label}", node_attr={'shape' : 'circle'}) as main:
            main.body.append(f'\t label=\"{graph_label}\"')
            main.body.append(f'\t color=\"white\"')
            with main.subgraph(name=name, node_attr={'shape' : 'circle'}) as s:
                s.body.append(f'\t label=\"{label}\"')
                s.body.append(f'\t color=\"black\"')
                for node in nodes:
                    if (node, level) in highlight.keys():
                        s.node(str(node)+level*" ",label=str(node),style="filled",fillcolor=highlight[(node, level)])
                    else:
                        s.node(str(node)+level*" ",label=str(node))
        return graph
    
    def create_edges(self, edges, graph, level, graph_type, highlight=dict()):
        level_x, level_y = level, level
        if graph_type == 0:
            level_y += 1
            
        for x,y in edges:
            if (x,y) in highlight.keys():
                graph.edge(str(x)+level_x*" ", str(y)+level_y*" ",penwidth="2",color=highlight[(x,y)])
            else:
                graph.edge(str(x)+level_x*" ", str(y)+level_y*" ",penwidth="2")
         
        return graph
    
    def fake_edges(self, graph, level, graph_type, highlight):
        level_x, level_y = level, level
        if graph_type == 0:
            level_y += 1
            
        if "dotted" in highlight.keys():
            for p, color in highlight["dotted"]:
                if graph_type == 2:
                    graph.edge(str(p[1])+level_x*" ", str(p[0])+level_y*" ",color=color,style="dotted")
                else:
                    graph.edge(str(p[0])+level_x*" ", str(p[1])+level_y*" ",color=color,style="dotted")
            highlight.pop("dotted")
        return graph, highlight
    
    def create_graph(self, label, values, graph_type):      
        if graph_type == 0:
            yield self.create_graph_binary(label, values)
        elif graph_type == 1:
            yield self.create_graph_homogeneous(label, values)
        elif graph_type == 2:
            yield self.create_graph_partial(label, values)
        elif graph_type == 3:
            yield self.create_graph_function(label, values)
            
    def create_graph_function(self, label, values):
        return self.create_graph_binary(label, values)
    
    def create_graph_binary(self, label, values):
        g = Digraph(node_attr={'shape': 'circle'})
        g.body.append(f'\t label=\"{label}\"')
        g.body.append('\t rankdir=LR ')
        level = 1
        for name, relation, highlight_node, highlight_edge in values:
            g = self.create_subgraph(f'cluster_{name}_dom', f"{name}-domain", \
                                     relation.get_domain(), g, name, level, highlight_node)
            g = self.create_subgraph(f'cluster_{name}_codom', f"{name}-codomain", \
                                     relation.get_codomain(), g, name, level+1, highlight_node)
            g = self.create_edges([(x,y) for x,y in relation.get_relation()],\
                                  g, level, 0, highlight_edge)
            g, highlight_edge = self.fake_edges(g, level, 0, highlight_edge)
            level += 2
        return g
    
    def create_graph_homogeneous(self, label, values):
        g = Digraph(node_attr={'shape': 'circle'})
        g.body.append(f'\t label=\"{label}\"')
        g.body.append('\t rankdir=BT')
        level = 1
        for name, relation, highlight_node, highlight_edge in values:
            g = self.create_subgraph(f'cluster_{name}', name, \
                                     relation.get_domain(), g, "", level, highlight_node)
            g = self.create_edges([(x,y) for x,y in relation.get_relation()],\
                                      g, level, 1, highlight_edge)
            g, highlight_edge = self.fake_edges(g, level, 1, highlight_edge)
            level += 1
        return g
    
    def create_graph_partial(self, label, values):
        g = Digraph(node_attr={'shape': 'circle'})
        g.body.append(f'\t label=\"{label}\"')
        g.body.append('\t rankdir=BT;')   
        g.body.append('\t edge [arrowhead="none"];')

        level = 1
        for name, relation, highlight_node, highlight_edge in values:         
            codom = relation.get_codomain()              
            g = self.create_subgraph(f'cluster_{name}', name, \
                                     relation.get_codomain(), g, "", level, highlight_node)
            dic = dict()
            for x in codom:
                dic[x] = {p for p in relation.is_out_relation_with(x)}
            for x in codom:
                h = dic[x]
                for s in dic.keys():
                    if x != s and s in dic[x]:
                        h = h - (dic[s] - {s})       
                g = self.create_edges([(x,p) for p in h if x!=p], g, level, 2, highlight_edge)
                g, highlight_edge = self.fake_edges(g, level, 2, highlight_edge)                          
            level += 1
        return g
    
    
          
    
    
    #--------------------------------GRAPH-----------------------------------------------------END

In [63]:
#r = BinaryRelation({('a',3),('b',3),('a',4),('b',4),('d',1)},{'a','b','c','d'},{1,2,3,4})
#r = BinaryRelation({(1,3),(2,3),(1,4),(2,4),(4,1)},{1,2,3,4},{1,2,3,4})
#universe = {1,2,3,4,'a','b','c','d'}
#variables = {'R':r}
#interpreter = Interpreter(variables,universe, True)

#g = interpreter.create_graph_binary(r, f"<<EXPLAIN - CONDITION>> \nRelation R is equal to relation S!", "R -> S")

#print(g.source)
#display(g)

In [64]:
#r = BinaryRelation({('a',3),('b',3),('a',4),('b',4)},{'a','b','c'},{1,2,3,4})
#print(r)

#universe = {1,2,3,4,'a','b','c'}
#variables = {'R':r}
#interpreter = Interpreter(variables,universe, True)
#print(r.matrix_representation())
#g = interpreter.create_matrix(r.matrix_representation(), "R is reflexive", \
                              #{(list(FiniteSet('b'))[0],list(FiniteSet(3))[0]):"green",('a',4):"green"})
#print(g.source)

#g.__next__()

In [65]:
import unittest

class TestNotebook_interpreter(unittest.TestCase):

    relation1 = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
    relation2 = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)
    relation3 = HomogeneousRelation({(1,1),(2,2),(3,3),(4,4)},{1,2,3,4}, True)
    universe1 = {1,2,3}
    
    def test_valid_interpreter(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertEqual(interpreter.get_variables(), variables)
        self.assertEqual(interpreter.get_universe(), self.universe1)
        
    def test_set_universe_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_universe = {1,2,3,4,'h','ch'}
        interpreter.set_universe(new_universe, True)
        self.assertEqual(interpreter.get_universe(), new_universe)
        
    def test_set_universe_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_universe = {2,3,4,'h','ch'}
        self.assertRaises(Exception, interpreter.set_universe,new_universe, True)
        
    def test_set_variables_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_variables =  {'R':self.relation1, 'S':self.relation1, 'T':self.relation2}
        interpreter.set_variables(new_variables, True)
        self.assertEqual(interpreter.get_variables(), new_variables)
        
    def test_set_variables_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        new_variables =  {'R':self.relation1, 'S':self.relation1, 'T':self.relation3}
        self.assertRaises(Exception, interpreter.set_variables,new_variables, True)
        
    def test_set_variable_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        interpreter.set_variable('S',self.relation2, True)
        new_variables =  {'R':self.relation1, 'S':self.relation2}
        self.assertEqual(interpreter.get_variables(), new_variables)
        
    def test_set_variable_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.set_variable,'T',self.relation2, True)
        
    def test_set_variable_not_valid_universe(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.set_variable,'R',self.relation3, True)
        
    def test_add_variable_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        interpreter.add_variable('T',self.relation2, True)
        new_variables =  {'R':self.relation1, 'S':self.relation1,'T':self.relation2}
        self.assertEqual(interpreter.get_variables(), new_variables)
        
    def test_add_variable_not_valid(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.add_variable,'R',self.relation2, True)
        
    def test_add_variable_not_valid_universe(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        self.assertRaises(Exception, interpreter.add_variable,'T',self.relation3, True)
        
    def test_interpret_formula_01(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        tree = interpreter.parse("R is transitive and S is transitive and R | S is not transitive")
        self.assertEqual(interpreter.interpret(tree), False)
        
    def test_interpret_formula_02(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        tree = interpreter.parse("R is transitive and S is transitive -> R | S is transitive")
        self.assertEqual(interpreter.interpret(tree), True)
        
    def test_interpret_formula_03(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)       
        U = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
        V = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)
        interpreter.add_variable('U',U, True)
        interpreter.add_variable('V',V, True)       
        tree = interpreter.parse("V is lattice and U|V is transitive ->  V is total_order")
        self.assertEqual(interpreter.interpret(tree), True)
        
    def test_node2string_01(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "R is transitive and S is transitive -> R | S is transitive"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_02(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "R is reflexive or S is not transitive -> R ^ S is not asymmetric"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_03(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "R is antisymmetric and S is transitive -> R - S is lattice"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_04(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "S is reflexive and not(R is antisymmetric and S is transitive) -> R ° S is not transitive"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_05(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "(S is reflexive and R is not reflexive) and not(R is antisymmetric and S is transitive) -> R <= S"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_06(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "(S is reflexive and R is not reflexive) and R is reflexive"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_07(self):
        variables = {'R':self.relation1, 'S':self.relation1}
        interpreter = Interpreter(variables,self.universe1, True)
        text = "S is reflexive and (R is not reflexive and R is reflexive)"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_08(self):
        domain = {'a','b','c','d'}
        codomain = {'a','b','c','d','e'}
        relation1 = Function({('c','c'),('a','b'),('b','d'),('d','a'),('e','a')}, codomain,domain, True)
        variables = {'R':relation1, 'S':relation1}
        interpreter = Interpreter(variables,domain|codomain, True)
        text = "S is bijective and (R is not surjective and R is injective)"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_node2string_09(self):
        domain = {'a','b','c','d'}
        codomain = {'a','b','c','d','e'}
        relation1 = Function({('c','c'),('a','b'),('b','d'),('d','a'),('e','a')}, codomain,domain, True)
        variables = {'R':relation1, 'S':relation1}
        interpreter = Interpreter(variables,domain|codomain, True)
        text = "S-1 and (R-1 is not surjective and R-1 is injective)"
        tree1 = interpreter.parse(text)
        tree2 = interpreter.parse(interpreter.node2string(tree1))
        self.assertEqual(tree1, tree2)
        
    def test_inverse(self):
        domain = {'a','b','c'}
        codomain = {1,2,3}
        relation1 = Function({('a',1),('b',2),('c',3)}, domain,codomain, True)
        variables = {'R':relation1}
        interpreter = Interpreter(variables,domain|codomain, True)
        text = "R-1"
        res1 = interpreter.interpret(interpreter.parse(text))
        relation1 = Function({(1,'a'),(2,'b'),(3,'c')}, codomain,domain, True)
        variables = {'S':relation1}
        interpreter = Interpreter(variables,domain|codomain, True)
        text = "S"
        res2 = interpreter.interpret(interpreter.parse(text))
        self.assertEqual(res1, res2)
    
    
        
unittest.main(argv=[''], verbosity=1, exit=False)

....................................................................................................................
----------------------------------------------------------------------
Ran 116 tests in 1.802s

OK


In [66]:
import unittest

class TestNotebook_explanator(unittest.TestCase):

    relation1 = HomogeneousRelation({(1,1),(2,2),(3,3),(1,2),(2,3)}, {1,2,3}, True)
    relation2 = HomogeneousRelation({(1,2),(2,1)}, {1,2,3}, True)
    relation3 = HomogeneousRelation({(1,1),(2,2),(3,3),(2,3)}, {1,2,3}, True)
    relation4 = HomogeneousRelation({(1,2)}, {1,2,3}, True)
    variables = {'A':relation1,
                 'B':relation1,
                 'C':relation1,
                 'D':relation1,
                 'E':relation1,
                 'F':relation2,
                 'R':relation3,
                 'S':relation4}
    interpreter = Interpreter(variables, {1,2,3}, True)

    condition1 = "A is symmetric and F is reflexive"
    condition2 = "A is not reflexive and (B is not reflexive or C is not reflexive)"
    condition3 = "(A is not reflexive and B is reflexive) -> (C is not reflexive and D is not reflexive)"
    condition4 = "(A is not reflexive and (B is not reflexive or C is not reflexive)) -> D is not reflexive"
    condition5 = "A is  reflexive -> B is reflexive"
    condition6 = "A is  reflexive -> (B is reflexive or C is not reflexive)"
    condition7 = "A is  reflexive -> (B is reflexive and C is reflexive)"
    condition8 = "R is transitive and S is transitive -> R | S is transitive"
    
    def test_explanator_01(self):
        generator = self.interpreter.analyze_all(self.condition4)
        counter = 0
        for p in generator:
            counter += 1
        self.assertEqual(counter, 8)
        
    def test_explanator_02(self):
        generator = self.interpreter.analyze_all(self.condition8)
        result = ""
        for p in generator:
            result += p
        self.assertEqual(p, "Good counterexample!")

unittest.main(argv=[''], verbosity=1, exit=False)

....................................................................................................................
----------------------------------------------------------------------
Ran 116 tests in 1.794s

OK


In [67]:
# relation3 = HomogeneousRelation({(1,1),(2,2),(3,3),(2,3)}, {1,2,3}, True)
# relation4 = HomogeneousRelation({(1,2)}, {1,2,3}, True)
# variables = {'R':relation3,
#              'S':relation4}
# interpreter = Interpreter(variables, {1,2,3}, True)
# condition8 = "(R is transitive and S is transitive and (R == S°R&S)) -> (R | S is transitive or (R != S))"
# generator = interpreter.analyze_all(condition8)
# for p in generator: print(p);

In [68]:
#print(condition8.pretty())

In [69]:
#relation1 = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
#relation2 = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)

#interpreter = Interpreter({'R':relation1,'S':relation2}, {1,2,3}, True)
# #interpreter.set_variables({'R':relation1,'X':HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3,4}, True)}, True) #Error

In [70]:
#tree = interpreter.parse("R is transitive and S is transitive -> R | S is transitive")

#print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
#print(f"[RESULT -> {interpreter.interpret(tree)}]")

In [71]:
# tree = interpreter.parser.parse("R is transitive and S is transitive and not R | S is transitive")

# print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
# print(f"[RESULT -> {interpreter.interpret(tree)}]")

In [72]:
# U = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
# V = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)

# interpreter.add_variable('U',U, True)
# interpreter.add_variable('V',V, True)
# tree = interpreter.parser.parse("V is lattice and U|V is transitive ->  V is total_order")

# print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
# print(f"[RESULT -> {interpreter.interpret(tree)}]")

# print(f"\nvariables:\n {interpreter.get_variables()}")

In [73]:
# V = PartialOrder({(1,1),(2,2),(3,3)}, {1,2,3},True)

# #interpreter.add_variable('V',V) #Error - already defined variable
# interpreter.set_variable('V',V,True)
# #interpreter.set_universe({1,2},True) #Error - not valid universe

# tree = interpreter.parser.parse("V != V ->  S is total_order")




# print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
# print(f"[RESULT -> {interpreter.interpret(tree)}]")

# print(f"\nvariables:\n {interpreter.get_variables('V','S')}")

In [74]:
#interpreter.get_variables()

In [75]:
#interpreter.get_universe()